# Catchment Boundaries

Notebook to create the files `CAMELS_DE_catchment_boundaries.geopackage` and `CAMELS_DE_catchment_boundaries.shp`.  
As well as `CAMELS_DE_gauging_stations.geopackage` and `CAMELS_DE_gauging_stations.shp`.  

We always use `EPSG:3035` as the coordinate reference system.

In [1]:
from glob import glob
import os
import pandas as pd
import geopandas as gpd

from camelsp import Station

In [2]:
# get camels_ids from hydromet timeseries
camels_ids = [camels_id.split("_")[-1].split(".csv")[0] for camels_id in glob("../output_data/camels_de/timeseries/*.csv")]

# sort camels_ids
camels_ids = sorted(camels_ids)

print(f"Total number of stations in CAMELS-DE v1: {len(camels_ids)}")

Total number of stations in CAMELS-DE v1: 1582


## Save catchment boundaries

Get all the catchment boundaries from the CAMELS-DE dataset and save them in a geopackage and shapefile.

In [15]:
catchments = []

for camels_id in camels_ids:
    # initialize station
    s = Station(camels_id)

    # get MERIT Hydro catchment
    catchment = s.get_catchment("merit_hydro")

    # from epsg:4326 to epsg:3035
    catchment = catchment.to_crs(epsg=3035)

    # add catchment to catchments
    catchments.append(catchment)

# concatenate all catchments into a single GeoDataFrame
catchments = gpd.pd.concat(catchments, ignore_index=True)
catchments.crs = "EPSG:3035"

# rename columns
catchments = catchments.rename(columns={"id": "gauge_id", "name": "gauge_name"})

# drop columns
catchments = catchments.drop(columns=["result", "area_calc", "area_reported"])

# # make directory if not exists
os.makedirs("../output_data/camels_de/CAMELS_DE_catchment_boundaries", exist_ok=True)
os.makedirs("../output_data/camels_de/CAMELS_DE_catchment_boundaries/catchments", exist_ok=True)

# save catchments
catchments.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/catchments/CAMELS_DE_catchments.shp")
catchments.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/catchments/CAMELS_DE_catchments.gpkg", driver="GPKG")

In [16]:
catchments

,gauge_id,gauge_name,geometry
0,DE110000,Kirchen-Hausen,"POLYGON ((4208675.183 2754027.132, 4208669.695..."
1,DE110010,Möhringen,"POLYGON ((4208675.183 2754027.132, 4208669.695..."
2,DE110020,Hundersingen,"POLYGON ((4225824.416 2753983.319, 4225761.576..."
3,DE110030,Berg,"POLYGON ((4225824.416 2753983.319, 4225761.576..."
4,DE110040,Achstetten,"POLYGON ((4318114.983 2781319.465, 4317928.853..."
...,...,...,...
1577,DEG10580,Unterbreizbach-Räsa,"POLYGON ((4312781.063 3061807.891, 4312604.312..."
1578,DEG10590,Vacha,"POLYGON ((4325316.084 3079045.275, 4325198.405..."
1579,DEG10600,Wasserthaleben,"POLYGON ((4369192.700 3128034.083, 4369076.135..."
1580,DEG10610,Weida,"POLYGON ((4468092.262 3058697.586, 4468094.862..."


## Save gauging station locations

In [4]:
# read station locations from topographic attributes
df = pd.read_csv("../output_data/camels_de/CAMELS_DE_topographic_attributes.csv")

# convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["gauge_easting"], df["gauge_northing"]))
gdf.crs = "EPSG:3035"

# make directory if not exists
os.makedirs("../output_data/camels_de/CAMELS_DE_catchment_boundaries/gauging_stations", exist_ok=True)

# save station locations
gdf.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/gauging_stations/CAMELS_DE_gauging_stations.shp")
gdf.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/gauging_stations/CAMELS_DE_gauging_stations.gpkg", driver="GPKG")

/tmp/ipykernel_140275/1604251584.py:12: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/stations/stations.shp")
